In [1]:
import time
import re
import json
import statistics as stats
import requests as req
import numpy as np
import pandas as pd
from unidecode import unidecode
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains as AC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
import tqdm
import feedparser
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [46]:
# Configurar opciones de Chrome
chrome_options = Options()

chrome_options.add_argument(r"user-data-dir=C:\\Users\\User\\AppData\\Local\\Google\\Chrome\\User Data")
chrome_options.add_argument("--remote-allow-origins=*")
chrome_options.add_experimental_option("useAutomationExtension", True) 
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_argument("user-data-dir=selenium-cookies")
chrome_options.add_argument('--headless')  # Habilitar si no queremos ver la ventana
chrome_options.add_experimental_option("detach", True)  # Esta opción corrige el error de cierre repentino
chrome_options.add_argument('--start-minimized')
chrome_options.add_argument('--disable-gpu')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-extensions')
chrome_options.add_argument('--disable-infobars')

In [2]:
#driver configuration
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False    # si True, no aperece la ventana (headless=no visible)
opciones.add_argument('--start-maximized')         # comienza maximizado
#opciones.add_argument('user-data-dir=selenium')    # mantiene las cookies
#opciones.add_extension('driver_folder/adblock.crx')       # adblocker
opciones.add_argument('--incognito')

In [3]:
PATH = 'driver/chromedriver'

driver = webdriver.Chrome()   # abre una ventana de chrome 
                            # una vez que he abierto una ventana, puedo trabajar siempre sobre esta misma.

driver.get('https://www.google.es')

In [4]:
url = 'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_object@@@pm:objectNode=http://museodelprado.es/items/object_1'

In [5]:

driver.get(url)

In [ ]:
//*[@id="ecidoc--p65_E36_p138_represents_object---pm--objectNode"]/ul[1]/li[1]/div

In [29]:
url = 'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_object@@@pm:objectNode=http://museodelprado.es/items/object_1'

In [30]:
driver.get(url)

In [26]:
pages = driver.find_elements(By.TAG_NAME, 'a')
pages

[<selenium.webdriver.remote.webelement.WebElement (session="8ecd939652a92e87f63ee99b631e80ef", element="65876CFE674351AB9D56F89578559EF7_element_721")>,
 <selenium.webdriver.remote.webelement.WebElement (session="8ecd939652a92e87f63ee99b631e80ef", element="65876CFE674351AB9D56F89578559EF7_element_723")>,
 <selenium.webdriver.remote.webelement.WebElement (session="8ecd939652a92e87f63ee99b631e80ef", element="65876CFE674351AB9D56F89578559EF7_element_728")>,
 <selenium.webdriver.remote.webelement.WebElement (session="8ecd939652a92e87f63ee99b631e80ef", element="65876CFE674351AB9D56F89578559EF7_element_729")>,
 <selenium.webdriver.remote.webelement.WebElement (session="8ecd939652a92e87f63ee99b631e80ef", element="65876CFE674351AB9D56F89578559EF7_element_730")>,
 <selenium.webdriver.remote.webelement.WebElement (session="8ecd939652a92e87f63ee99b631e80ef", element="65876CFE674351AB9D56F89578559EF7_element_731")>,
 <selenium.webdriver.remote.webelement.WebElement (session="8ecd939652a92e87f63ee9

In [27]:
for e in pages:
    print(e.get_attribute("href"))

https://www.museodelprado.es/
https://entradas.museodelprado.es/
https://www.museodelprado.es/en/the-collection/art-works
https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_object@@@pm:objectNode=http://museodelprado.es/items/object_1#modal-buscador
None
https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_object@@@pm:objectNode=http://museodelprado.es/items/object_1#
https://www.museodelprado.es/en/the-collection/art-works
https://www.museodelprado.es/visita-el-museo
https://www.museodelprado.es/visita/horarios-y-tarifas
https://www.museodelprado.es/visita/prepara-tu-visita
https://www.museodelprado.es/visita/grupos
https://www.museodelprado.es/visita/accesible
https://www.museodelprado.es/visita/recomendaciones
https://www.museodelprado.es/visita/como-llegar
https://www.museodelprado.es/coleccion
https://www.museodelprado.es/coleccion/obras-de-arte
https://www.museodelprado.es/coleccion/linea-del-tiempo
https://www.museo

In [ ]:
//*[@id="ecidoc--p65_E36_p138_represents_object---pm--objectNode"]/ul[1]/li[1]/div/p[2]/a

In [ ]:
//div[@class="desplegable mas-menos on"]/h2/a

In [4]:
url = 'https://www.museodelprado.es/coleccion/obras-de-arte?'

In [5]:
driver.get(url)

In [14]:
#ESTE ES EL BUENO! CONSEGUIDO!
# Obtener los elementos que deseas procesar
elementos = driver.find_elements(By.XPATH, '/html/body/main/section[3]/div[3]/div[15]/ul[1]/li/div/p/a')
# Ahora puedes usar la lista 'elementos' en tu bucle
hrefs = []

for elemento in elementos:
    try:
        # Obtén el enlace (href) dentro del elemento
        href = elemento.get_attribute('href')
        hrefs.append(href)
    except NoSuchElementException:
        # Manejo de excepción si no se encuentra el elemento href
        print("No se encontró el elemento href en:", elemento.text)

# Mostrar los resultados con print en lugar de return
print(hrefs)



['https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_object@@@pm:objectNode=http://museodelprado.es/items/object_1', 'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_object@@@pm:objectNode=http://museodelprado.es/items/object_1&ecidoc:p65_E36_p138_represents_object@@@pm:objectNode=http://museodelprado.es/items/object_353', 'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_object@@@pm:objectNode=http://museodelprado.es/items/object_1&ecidoc:p65_E36_p138_represents_object@@@pm:objectNode=http://museodelprado.es/items/object_354', 'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_object@@@pm:objectNode=http://museodelprado.es/items/object_1&ecidoc:p65_E36_p138_represents_object@@@pm:objectNode=http://museodelprado.es/items/object_355', 'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_object@@@pm:objectNode=http:

In [15]:
for href in hrefs:
    print(href)

https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_object@@@pm:objectNode=http://museodelprado.es/items/object_1
https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_object@@@pm:objectNode=http://museodelprado.es/items/object_1&ecidoc:p65_E36_p138_represents_object@@@pm:objectNode=http://museodelprado.es/items/object_353
https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_object@@@pm:objectNode=http://museodelprado.es/items/object_1&ecidoc:p65_E36_p138_represents_object@@@pm:objectNode=http://museodelprado.es/items/object_354
https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_object@@@pm:objectNode=http://museodelprado.es/items/object_1&ecidoc:p65_E36_p138_represents_object@@@pm:objectNode=http://museodelprado.es/items/object_355
https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_object@@@pm:objectNode=http://museodelprad

In [7]:
url = 'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_object@@@pm:objectNode=http://museodelprado.es/items/object_1'

In [8]:
driver.get(url)

In [22]:

# Obtener los elementos que deseas procesar
elementos = driver.find_elements(By.XPATH, './/ul[1]/li[1]/div/p[1]/a') 

# Ahora puedes usar la lista 'elementos' en tu bucle
hrefs = []

for elemento in elementos:
    try:
        # Obtén el enlace (href) dentro del elemento
        href = elemento.get_attribute('href')
        hrefs.append(href)
    except NoSuchElementException:
        # Manejo de excepción si no se encuentra el elemento href
        print("No se encontró el elemento href en:", elemento.text)

# Mostrar los resultados con print en lugar de return
print(hrefs)

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=120.0.6099.199)
Stacktrace:
0   chromedriver                        0x00000001024984dc chromedriver + 4162780
1   chromedriver                        0x0000000102490664 chromedriver + 4130404
2   chromedriver                        0x00000001020e7bc0 chromedriver + 293824
3   chromedriver                        0x00000001020cf0b0 chromedriver + 192688
4   chromedriver                        0x00000001020cefd8 chromedriver + 192472
5   chromedriver                        0x000000010216d2b0 chromedriver + 840368
6   chromedriver                        0x00000001021216bc chromedriver + 530108
7   chromedriver                        0x0000000102122930 chromedriver + 534832
8   chromedriver                        0x000000010245de08 chromedriver + 3923464
9   chromedriver                        0x00000001024623dc chrome

In [20]:
for href in hrefs:
    print(href)

https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_object@@@pm:objectNode=http://museodelprado.es/items/object_1#
https://www.museodelprado.es/en/the-collection/art-works
https://www.museodelprado.es/visita-el-museo
https://www.museodelprado.es/visita/horarios-y-tarifas
https://www.museodelprado.es/visita/prepara-tu-visita
https://www.museodelprado.es/visita/grupos
https://www.museodelprado.es/visita/accesible
https://www.museodelprado.es/visita/recomendaciones
https://www.museodelprado.es/visita/como-llegar
https://www.museodelprado.es/coleccion
https://www.museodelprado.es/coleccion/obras-de-arte
https://www.museodelprado.es/coleccion/linea-del-tiempo
https://www.museodelprado.es/coleccion/pintura-espanola
https://www.museodelprado.es/coleccion/pintura-italiana
https://www.museodelprado.es/coleccion/pintura-flamenca
https://www.museodelprado.es/coleccion/pintura-sxix
https://www.museodelprado.es/coleccion/dibujos-estampas-y-fotografia
https://www.muse

In [16]:

# Obtener los elementos que deseas procesar
elementos = driver.find_elements(By.XPATH, './/ul[1]/li[1]/div/p[1]/a')


# Ahora puedes usar la lista 'elementos' en tu bucle
hrefs = []

for elemento in elementos:
    try:
        # Obtén el enlace (href) dentro del elemento
        href = elemento.find_element(By.XPATH, './/ul[1]/li[1]/div/p[1]/a').get_attribute('href')
        hrefs.append(href)
    except NoSuchElementException:
        # Manejo de excepción si no se encuentra el elemento href
        print("No se encontró el elemento href en:", elemento.text)

return hrefs


No se encontró el elemento href en: Fiestas y Ceremonias (8)
No se encontró el elemento href en: 


SyntaxError: 'return' outside function (3208439040.py, line 17)

In [ ]:
hrefs = []

for elemento in elementos:
    try:
        # Obtén el enlace (href) dentro del elemento
        href = elemento.find_element(By.XPATH, './/ul[1]/li[1]/div/p[1]/a').get_attribute('href')
        hrefs.append(href)
    except NoSuchElementException:
        # Manejo de excepción si no se encuentra el elemento href
        print("No se encontró el elemento href en:", elemento.text)

return hrefs


In [17]:
from selenium.common.exceptions import TimeoutException

def obtener_hrefs(driver, xpath):
    # Espera implícita para asegurarse de que la página esté completamente cargada
    driver.implicitly_wait(10)

    # Espera explícita para asegurarse de que los elementos estén presentes antes de buscarlos
    wait = WebDriverWait(driver, 10)

    try:
        # Intentar encontrar y hacer clic en el botón de aceptar cookies
        aceptar = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="onetrust-accept-btn-handler"]')))
        aceptar.click()
        print('Cookies aceptadas correctamente.')
    except TimeoutException:
        print('No se encontró el botón para aceptar cookies.')

    try:
        elementos = wait.until(EC.presence_of_all_elements_located((By.XPATH, xpath)))
        hrefs = []

        for elemento in elementos:
            try:
                # Obtén el enlace (href) dentro del elemento
                href = elemento.find_element(By.XPATH, './/ul[1]/li[1]/div/p[1]/a').get_attribute('href')
                hrefs.append(href)
            except NoSuchElementException:
                # Manejo de excepción si no se encuentra el elemento href
                print("No se encontró el elemento href en:", elemento.text)

        return hrefs

    except TimeoutException:
        print(f"No se pudieron encontrar elementos con el XPath: {xpath}")



# Obtener hrefs
url = 'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_object@@@pm:objectNode=http://museodelprado.es/items/object_1'
hrefs = obtener_hrefs(driver, xpath_elementos)

# Imprimir los hrefs obtenidos
print(hrefs)




No se encontró el botón para aceptar cookies.
No se pudieron encontrar elementos con el XPath: /ul[1]/li[1]/div/p[1]
None


In [15]:
def obtener_hrefs(driver, xpath):
    # Espera implícita para asegurarse de que la página esté completamente cargada
    driver.implicitly_wait(10)

    
    
    # Espera explícita para asegurarse de que los elementos estén presentes antes de buscarlos
    wait = WebDriverWait(driver, 10)
    elementos = wait.until(EC.presence_of_all_elements_located((By.XPATH, xpath)))
    
    hrefs = []
    
    for elemento in elementos:
        try:
            # Obtén el enlace (href) dentro del elemento
            href = elemento.find_element(By.XPATH, './/a').get_attribute('href')
            hrefs.append(href)
        except NoSuchElementException:
            # Manejo de excepción si no se encuentra el elemento href
            print("No se encontró el elemento href en:", elemento.text)
    
    return hrefs

# Ejemplo de uso
url = 'https://www.museodelprado.es/coleccion/obras-de-arte?ecidoc:p65_E36_p138_represents_object@@@pm:objectNode=http://museodelprado.es/items/object_1'
xpath_elementos = '//*[@id="ecidoc--p65_E36_p138_represents_object---pm--objectNode"]/ul[1]/li'

# Obtener hrefs
hrefs = obtener_hrefs(driver, xpath_elementos)

# Imprimir los hrefs obtenidos
print(hrefs)

TimeoutException: Message: 
